# 파라메터 서칭
tree base 모델은 설정 가능한 파라메터의 조합에 따라 모델 예측력 차이가 큰 특징을 가지고 있습니다.  
특히, Xgboost 모델의 경우 파라메터 설정에 따른 모델 예측력 차이가 굉장히 크기에 꼭 파라메터 서칭을 진행해주셔야 합니다.  
간단한 문법을 통해 파라메터 서칭을 진행 해보겠습니다.

## 파이썬 기본 문법으로 파라메터 서칭

In [5]:
# 필요 모듈 import
import pandas as pd
from itertools import product
from sklearn.model_selection import train_test_split

In [3]:
# 데이터셋 로딩
df = pd.read_csv('./data/boston.csv')

In [4]:
# 타겟 데이터 분할
y = df['y']
X = df.drop('y', axis=1)

In [6]:
# 학습 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [19]:
# 검증 데이터 분할
X_train2, X_val, y_train2, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

In [20]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score

In [21]:
# 파라메터 서칭
# randomforest 서칭 해야할 파라메터 max_depth, n_estimators
depth = [3, 5, 7, 9]
est = [100, 300, 500, 700, 900, 1000]

best_score = 0 # 아주 작은 값으로 초기화
best_param = 0

for param in list(product(depth, est)):
    print(f'{param} 서칭중')
    model = RandomForestRegressor(max_depth=param[0], n_estimators=param[1], random_state=42)
    model.fit(X_train2, y_train2)
    pred = model.predict(X_val)
    r2 = r2_score(y_val, pred)
    if r2 > best_score:
        print('최적 모델 갱신!')# 최댁값 찾는 알고리즘
        best_score = r2
        best_param = param

print('모델 학습 완료')   

(3, 100) 서칭중
최적 모델 갱신!
(3, 300) 서칭중
(3, 500) 서칭중
(3, 700) 서칭중
(3, 900) 서칭중
(3, 1000) 서칭중
(5, 100) 서칭중
최적 모델 갱신!
(5, 300) 서칭중
(5, 500) 서칭중
(5, 700) 서칭중
(5, 900) 서칭중
(5, 1000) 서칭중
(7, 100) 서칭중
최적 모델 갱신!
(7, 300) 서칭중
(7, 500) 서칭중
(7, 700) 서칭중
(7, 900) 서칭중
(7, 1000) 서칭중
(9, 100) 서칭중
(9, 300) 서칭중
(9, 500) 서칭중
(9, 700) 서칭중
(9, 900) 서칭중
(9, 1000) 서칭중
모델 학습 완료


In [23]:
# 위 파라메터 서칭 코드로 찾은 최적 모델의 평가지표와 파라메터 확인
best_score, best_param

(0.7132323588740164, (7, 100))

In [24]:
# 최적 모델로 모델 다시 학습 및 평가
best_model = RandomForestRegressor(random_state=42, max_depth=7, n_estimators=100)
best_model.fit(X_train, y_train)
best_pred = best_model.predict(X_test)
print(r2_score(y_test, best_pred))

0.865436646031359


## sklearn GridSearchCV
sklearn 패키지에는 위의 파라메터 서칭 과정을 간편하게 진행 할 수 있도록 GridSearchCV 방법론을 제공합니다.  
기존 파라메터 서칭과 함께 cross validation 과정을 추가하여 데이터 분할에 강건한 모델을 선택할 수 있도록 제작 되었습니다.

In [ ]:
''' xgboost 파라메터 서칭 목록
max_depth : classification [3, 5, 7, 9], regression [7, 9, 11, 13, 15]
n_estimators : 부스팅 트리 갯수, [100, 300, 500, 700, 1000, 2000, 3000]
learning_rate : round별 학습률 [0.001, 0.003, 0.01, 0.03]
subsample : 부트스트랩 샘플 비율 [0.6, 0.7, 0.8]
colsample_bytree : 부트스트랩 컬럼 비율 [0.6, 0.7, 0.8]
설명변수 갯수가 많은 경우 아래 파라메터도 서칭
reg_alpha : L1, lasso [1, 3, 5, 7, 9]
reg_lambda : L2, ridge, [1, 3, 5, 7, 9]
'''

In [25]:
# 그리드 서치 import
from sklearn.model_selection import GridSearchCV

In [26]:
# 그리드 서치를 위한 모델 및 파라메터 준비
est = RandomForestRegressor(random_state=42)
param = {
    'max_depth':[3, 5, 7, 9],
    'n_estimators':[100, 300, 500, 700]
}

In [27]:
# 그리드 서치 모델 생성
grid = GridSearchCV(estimator=est, 
                    param_grid=param, 
                    scoring='neg_root_mean_squared_error',
                    verbose=2,
                    cv=5)
'''
estimator : 모델 딕셔너리
param_grid : 파라메터 딕셔너리
scoring=None : 평가방법
n_jobs=None : 학습에 사용할 컴퓨터 코어 갯수 
verbose=0 : 리포트 형식 0, 1, 2

scoring 참고
https://scikit-learn.org/stable/modules/model_evaluation.html
'''

'\nestimator : 모델 딕셔너리\nparam_grid : 파라메터 딕셔너리\nscoring=None : 평가방법\nn_jobs=None : 학습에 사용할 컴퓨터 코어 갯수 \nverbose=0 : 리포트 형식 0, 1, 2\n\nscoring 참고\nhttps://scikit-learn.org/stable/modules/model_evaluation.html\n'

In [28]:
# grid 학습
grid.fit(X_train, y_train)

Fitting 5 folds for each of 16 candidates, totalling 80 fits
[CV] END ......................max_depth=3, n_estimators=100; total time=   0.1s
[CV] END ......................max_depth=3, n_estimators=100; total time=   0.1s
[CV] END ......................max_depth=3, n_estimators=100; total time=   0.1s
[CV] END ......................max_depth=3, n_estimators=100; total time=   0.1s
[CV] END ......................max_depth=3, n_estimators=100; total time=   0.1s
[CV] END ......................max_depth=3, n_estimators=300; total time=   0.6s
[CV] END ......................max_depth=3, n_estimators=300; total time=   0.6s
[CV] END ......................max_depth=3, n_estimators=300; total time=   0.6s
[CV] END ......................max_depth=3, n_estimators=300; total time=   0.6s
[CV] END ......................max_depth=3, n_estimators=300; total time=   0.6s
[CV] END ......................max_depth=3, n_estimators=500; total time=   1.1s
[CV] END ......................max_depth=3, n_es

GridSearchCV(cv=5, estimator=RandomForestRegressor(random_state=42),
             param_grid={'max_depth': [3, 5, 7, 9],
                         'n_estimators': [100, 300, 500, 700]},
             scoring='neg_root_mean_squared_error', verbose=2)

In [ ]:
# 최적 모델 및 파라메터 확인


In [ ]:
# 최적 모델로 모델 다시 학습 및 평가
